In [1]:
!pip install requests -q
!pip install torch -q
!pip install pillow -q
!pip install transformers>="4.45.0" --upgrade

In [5]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",

)
processor = AutoProcessor.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
#url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
#url = "https://www.researchgate.net/publication/340062988/figure/fig2/AS:871194861502466@1584720449974/Structural-damage-to-elevated-houses-due-to-the-wind-during-Hurricane-Katrina-2005-FEMA.jpg"
#url = "https://lisamillerassociates.com/wp-content/uploads/2020/03/FEMA-MAT-Figure-4-11-Post-FBC-house-with-signif-structural-damage.png"
#url = "https://images.e-flux-systems.com/Image_0_Effects_of_Hurricane_Charley_from_FEMA_Photo_Library_7.jpg,1600"
url = "https://img.freepik.com/free-photo/modern-house-architectural-design-photography_1409-6516.jpg?t=st=1727709206~exp=1727712806~hmac=193e2ee44a329b8956e64a1b1433ffb94ba885439ed6d8086e01839fab0582a9&w=1480"

image = Image.open(requests.get(url, stream=True).raw)

#instruction = "Describe this picture."
instruction = "Is there structural damage in this picture? If so, describe the damage. If not, say 'No structural damage detected."
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]

input_text = processor.apply_chat_template(
    messages, add_generation_prompt=True,
)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to(model.device)
output = model.generate(**inputs, max_new_tokens=70)

print(processor.decode(output[0][inputs["input_ids"].shape[-1]:]))

The image does not show any visible structural damage. The house appears to be well-maintained, with no signs of cracks, water damage, or other issues that could indicate structural problems.<|eot_id|>
